In [4]:
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP

# 生成 RSA 密钥对
key = RSA.generate(2048)
private_key = key.export_key()
public_key = key.publickey().export_key()

# 保存密钥对到文件
with open('private_key.pem', 'wb') as private_file:
    private_file.write(private_key)

with open('public_key.pem', 'wb') as public_file:
    public_file.write(public_key)

# 加载密钥对
with open('private_key.pem', 'rb') as private_file:
    private_key = RSA.import_key(private_file.read())

with open('public_key.pem', 'rb') as public_file:
    public_key = RSA.import_key(public_file.read())

# 加密消息
message = b"Hello, RSA!"
cipher = PKCS1_OAEP.new(public_key)
encrypted_message = cipher.encrypt(message)
print("Encrypted Message:", encrypted_message)

# 解密消息
cipher = PKCS1_OAEP.new(private_key)
decrypted_message = cipher.decrypt(encrypted_message)
print("Decrypted Message:", decrypted_message.decode('utf-8'))


Encrypted Message: b"\x9eF\x03\x9e\xba\x10\xac\xb3\xd1\x02\x08\t\xe9e\xe2\x07]\xa9k0\xda\xd2\xb7\x12\x92X\x19}NS\xef!f\xfeQO\xb2a\xc2S\x12\x83\xf3\xcf\x0f\x1d&'\x85\x83\xce\xfc\xa4\xc9\xc4\xb0\x17\xc2\x03\xd6X\x10*\xecL\x1bB0}\xb0O\xe8\xdf\xa0\xda\x83\x11\xe8\x10\xbbpV\xbb\x89f\xb5\xd6\xc0\x0f\xd2QJ\xd4\xcf \xcclHz\xc4\xa4Ap\xbcY\xda1t\xfa\x12^g\xac\xbc\x06\xb4\x86o|\xb60\x9b0I\nD{\x9d\xc6\xe0Y7\xf8\x17\xd1EP\x83a\xda\x906\x00H\xcc2N\x95\xcf\xd5\xfc(\x1b\xa9\xbb\xda\xc0D\x1b8\xc7\xf0\x1b\x9a\xbc\xc1\xa5\xeeI<\x02i\xc9\x97\xd8Y\xd8\xefV\x8d\x19A\xdb\x93\xe1\xaac9\x98\x01\xc8\x97\xd4KS\xb8\xdbt~\xa44\xc2\xd1\x97\x90\x9221J=\x82\xd8:\x80\xc2\xfd\xc0\xfcrEE\xe9\xf7\xa1\x06\xfb\xe7\x82\xf9\n\xaf,:\xf6eJ\xe3\xddg\x11a\xf4u_`\x01T\x8bY\x9c\x15[\xbb\x7f\xe6\x1f"
Decrypted Message: Hello, RSA!


In [5]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad

# 生成随机密钥
key = get_random_bytes(16)
print("AES Key:", key)

# 创建 AES 密钥对象
cipher = AES.new(key, AES.MODE_CBC)

# 加密消息
message = b"Hello, AES!"
ciphertext = cipher.encrypt(pad(message, AES.block_size))
print("Encrypted Message:", ciphertext)

# 解密消息
decipher = AES.new(key, AES.MODE_CBC, iv=cipher.iv)
decrypted_message = unpad(decipher.decrypt(ciphertext), AES.block_size)
print("Decrypted Message:", decrypted_message.decode('utf-8'))


AES Key: b'*\xe6\x00Gw\x15\xc4C`\xca\xf9\xad\xb35FP'
Encrypted Message: b'\xc2\xe5\xef\xe9\xe79\x01\xe3\x01\xc7\x8a%\xbc\x18\x12\x12'
Decrypted Message: Hello, AES!


In [6]:
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad
from myhttp.server import HTTPServerEncryptedClass


# 模拟服务器客户端加密通信

# 1. 客户端向服务器进行握手，希望进行加密通信。

# 2. 服务器收到请求，生成 RSA 公钥、RSA 私钥，将 RSA 公钥明文传输给客户端

http_server_encrypted = HTTPServerEncryptedClass()
rsa_public_key = http_server_encrypted.rsa_public_key
print("RSA Public Key:", rsa_public_key)

# 3. 客户端收到 RSA  公钥，生成 AES 秘钥，使用  RSA  公钥加密后，将密文传输给服务器。

aes_key = get_random_bytes(16)
aes_iv = get_random_bytes(16)
print("AES Key:", aes_key)
print("AES IV:", aes_iv)
cipher = PKCS1_OAEP.new(RSA.import_key(rsa_public_key))
encrypted_message = cipher.encrypt(aes_key + aes_iv)
print("Encrypted Message:", encrypted_message)

# 4. 服务器收到密文，使用 RSA 私钥解密，得到 AES 秘钥

decrypted_message = http_server_encrypted.rsa_decrypt(encrypted_message)
print("Decrypted Message:", decrypted_message)
http_server_encrypted.set_aes_key(decrypted_message[:16], decrypted_message[16:])

# # 5. 服务器和客户端后续的body，采用 AES 秘钥进行加密解密。

client_ases_cipher = AES.new(aes_key, AES.MODE_CBC, iv=aes_iv)
client_ases_decipher = AES.new(aes_key, AES.MODE_CBC, iv=aes_iv)

text_client_to_server = b"Hello, Server Mess1!"
ciphertext = client_ases_cipher.encrypt(pad(text_client_to_server, AES.block_size))
print("Encrypted Message:", ciphertext)
decrypted_message = http_server_encrypted.aes_decrypt(ciphertext)
print("Decrypted Message:", decrypted_message)

text_server_to_client = b"Hello, Client Mess1!"
ciphertext = http_server_encrypted.aes_encrypt(text_server_to_client)
print("Encrypted Message:", ciphertext)
decrypted_message = unpad(client_ases_decipher.decrypt(ciphertext), AES.block_size)
print("Decrypted Message:", decrypted_message.decode('utf-8'))

text_client_to_server = b"Hello, Server Mess2!"
ciphertext = client_ases_cipher.encrypt(pad(text_client_to_server, AES.block_size))
print("Encrypted Message:", ciphertext)
decrypted_message = http_server_encrypted.aes_decrypt(ciphertext)
print("Decrypted Message:", decrypted_message)

text_server_to_client = b"Hello, Client Mess2!"
ciphertext = http_server_encrypted.aes_encrypt(text_server_to_client)
print("Encrypted Message:", ciphertext)
decrypted_message = unpad(client_ases_decipher.decrypt(ciphertext), AES.block_size)
print("Decrypted Message:", decrypted_message.decode('utf-8'))


text_client_to_server = b"Hello, Server Mess3!"
ciphertext = client_ases_cipher.encrypt(pad(text_client_to_server, AES.block_size))
print("Encrypted Message:", ciphertext)
decrypted_message = http_server_encrypted.aes_decrypt(ciphertext)
print("Decrypted Message:", decrypted_message)

text_server_to_client = b"Hello, Client Mess3!"
ciphertext = http_server_encrypted.aes_encrypt(text_server_to_client)
print("Encrypted Message:", ciphertext)
decrypted_message = unpad(client_ases_decipher.decrypt(ciphertext), AES.block_size)
print("Decrypted Message:", decrypted_message.decode('utf-8'))

RSA Public Key: b'-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAu9g/NYQVT9RTBeAQB922\nlo75m9lvgRmlFKGNlhzZPaXPLfdwpvM+h6he2vhRsldrty6sA636TK7+hRyURtAY\nSVIEaP/nyhQO3DQARAom6iPhMSxnB/1iUUZzjFnYiNmQGLfxFdPZR34WISSdvOmv\nyX/NjJ724/RG7iBD8YEbmEYV8kxDcVubb3UT9pRr0Isq9ANCCkSTRuOMp+N1w5KI\n+kTgU/UyMPYq0NHbWIVbemWZgOOjDEJFCQSubk8q4wQVegTT+6VOx5pudmzRJniB\nLtM3mHVloQlTkWmgHTtNJFYb3TMpHlcKae4KGzVkMybpSc+36FzeLfV6tDzc3aaK\nqQIDAQAB\n-----END PUBLIC KEY-----'
AES Key: b'\xdfm\xa43X6>\xa3\x04Y6p\xdc\rhO'
AES IV: b'O\xad\xe9\x89\xba\x8c\xa2\xc2Mb\xbfN\x04\xff\xf5\xe9'
Encrypted Message: b"s\xa39{ojE\xfd\xaf=s\xd6E\xf1\x88\x9c\xbe4\x11\x87\x16\x86AmB:O\xd6$\x85nn\xbe\xc7WBo:g\xc3\x86\x9ad\xe2\xe3\xc3\x8c\xddpN\x17'\xef_%\xd3\x98/9\x85\xaa\xed\x0f\x1a+\x80\xd2\xf2\xc4\xef\x8eC\x90\x9deR1\x03\x9d\x91[\xcc\xe87\xf3\xc1G\xaem\x97\xfa4=D\xde\xa8\xc1\xb3j\xc7&\xcf\x07\xb8C+\xd1\xa0\x1d\x93C{xj\xc8\xd1\x85m~\xa8t\x8d\xd6r\xabw\xf2\n\x94>\x8a\xea\xf61\x1c(:?\x15\xbf\x9e\xc7\x9d-C\xa